In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('./train.csv')

In [2]:
N_USERS = 1216082

In [3]:
%%time

THRESHOLD = 25

users = {}
edges = {}
for row in df.itertuples():
    users[row[1]] = row[3]
    users[row[2]] = row[4]
    if row[5] + row[7] < THRESHOLD:
        continue
    if row[1] not in edges:
        edges[row[1]] = set()
    edges[row[1]].add(row[2])
    if row[2] not in edges:
        edges[row[2]] = set()
    edges[row[2]].add(row[1])

CPU times: user 16.3 s, sys: 408 ms, total: 16.7 s
Wall time: 17.9 s


In [4]:
def jaccard(set_1, set_2):
    return len(set_1 & set_2)/len(set_1 | set_2)

In [7]:
%%time

MIN_SIMILARITY = 0.5

with open('./submission.csv', 'w') as f:
    for i in range(N_USERS):
        candidates = set([candidate
                          for g in edges.get(i, set())
                          for candidate in edges.get(g, set())]) - edges.get(i, set()) - set([i])
        if users.get(i, 0) != 0:
            candidates = set([x for x in candidates if users[x] == 0])
        prediction = set()
        for v in candidates:
            j = jaccard(edges[i], edges[v])
            if j >= MIN_SIMILARITY:
                prediction.add(v)
        f.write(','.join([str(x) for x in [i] + list(candidates)]) + '\n')

CPU times: user 13.7 s, sys: 583 ms, total: 14.3 s
Wall time: 14.8 s
